<a href="https://colab.research.google.com/github/anjalipatel101/CPSC483_Project/blob/frontend_branch/version1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:


Open In Colab

from google.colab import drive
drive.mount('/gdrive')
!ln -s "/gdrive/MyDrive/481_AI_project/templates" "/content/templates"
!ln -s "/gdrive/MyDrive/481_AI_project/models" "/content/models"



Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
ln: failed to create symbolic link '/content/templates/templates': File exists
ln: failed to create symbolic link '/content/models/models': File exists


# Backend/UI imports
!pip install flask_ngrok
!pip install pyngrok==4.1.1
!pip install flask_cors


Requirement already satisfied: flask_ngrok in /usr/local/lib/python3.10/dist-packages (0.0.25)
Requirement already satisfied: Flask>=0.8 in /usr/local/lib/python3.10/dist-packages (from flask_ngrok) (2.2.5)
Requirement already satisfied: requests in /usr/local/lib/python3.10/dist-packages (from flask_ngrok) (2.31.0)
Requirement already satisfied: Werkzeug>=2.2.2 in /usr/local/lib/python3.10/dist-packages (from Flask>=0.8->flask_ngrok) (3.0.1)
Requirement already satisfied: Jinja2>=3.0 in /usr/local/lib/python3.10/dist-packages (from Flask>=0.8->flask_ngrok) (3.1.2)
Requirement already satisfied: itsdangerous>=2.0 in /usr/local/lib/python3.10/dist-packages (from Flask>=0.8->flask_ngrok) (2.1.2)
Requirement already satisfied: click>=8.0 in /usr/local/lib/python3.10/dist-packages (from Flask>=0.8->flask_ngrok) (8.1.7)
Requirement already satisfied: charset-normalizer<4,>=2 in /usr/local/lib/python3.10/dist-packages (from requests->flask_ngrok) (3.3.2)
Requirement already satisfied: idna<4,>=2.5 in /usr/local/lib/python3.10/dist-packages (from requests->flask_ngrok) (3.6)
Requirement already satisfied: urllib3<3,>=1.21.1 in /usr/local/lib/python3.10/dist-packages (from requests->flask_ngrok) (2.0.7)
Requirement already satisfied: certifi>=2017.4.17 in /usr/local/lib/python3.10/dist-packages (from requests->flask_ngrok) (2023.11.17)
Requirement already satisfied: MarkupSafe>=2.0 in /usr/local/lib/python3.10/dist-packages (from Jinja2>=3.0->Flask>=0.8->flask_ngrok) (2.1.3)
Requirement already satisfied: pyngrok==4.1.1 in /usr/local/lib/python3.10/dist-packages (4.1.1)
Requirement already satisfied: future in /usr/local/lib/python3.10/dist-packages (from pyngrok==4.1.1) (0.18.3)
Requirement already satisfied: PyYAML in /usr/local/lib/python3.10/dist-packages (from pyngrok==4.1.1) (6.0.1)
Requirement already satisfied: flask_cors in /usr/local/lib/python3.10/dist-packages (4.0.0)
Requirement already satisfied: Flask>=0.9 in /usr/local/lib/python3.10/dist-packages (from flask_cors) (2.2.5)
Requirement already satisfied: Werkzeug>=2.2.2 in /usr/local/lib/python3.10/dist-packages (from Flask>=0.9->flask_cors) (3.0.1)
Requirement already satisfied: Jinja2>=3.0 in /usr/local/lib/python3.10/dist-packages (from Flask>=0.9->flask_cors) (3.1.2)
Requirement already satisfied: itsdangerous>=2.0 in /usr/local/lib/python3.10/dist-packages (from Flask>=0.9->flask_cors) (2.1.2)
Requirement already satisfied: click>=8.0 in /usr/local/lib/python3.10/dist-packages (from Flask>=0.9->flask_cors) (8.1.7)
Requirement already satisfied: MarkupSafe>=2.0 in /usr/local/lib/python3.10/dist-packages (from Jinja2>=3.0->Flask>=0.9->flask_cors) (2.1.3)


!ngrok authtoken 2ZE5sJ2XtQ8GReACzTzcrLyGrAB_4LPopJEQ8Fs8HdQbyFReV # cardinal sin to have the auth token here but no time


Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


# Machine learning imports
import tensorflow as tf
import numpy as np
new_model = tf.keras.models.load_model('/content/models/hisA_model.keras')
# class_names = new_model.class_names








from flask import Flask, render_template
from flask_ngrok import run_with_ngrok
from flask import jsonify, request
from flask_cors import CORS

app = Flask(__name__)
CORS(app)
run_with_ngrok(app)

@app.route("/")
def hello():
  data = {
      "title": "Animal Chatter: Cat and Dog breed prediction"
  }

  return render_template('index.html', **data)

@app.route("/api/v1/predict", methods=["POST"])
def predict_breed():
  # check if the request sent over an imag

  if 'image' in request.files:
    req_image = request.files['image']
    req_image.save(req_image.filename)
    image = tf.keras.utils.load_img(f"/content/{req_image.filename}",target_size=(180,180))
    image = tf.keras.utils.img_to_array(image)
    image = image/255.0
    prediction_image=np.array(image)
    prediction_image= np.expand_dims(image, axis=0)
    weight_array = new_model.predict(prediction_image)
    score = tf.nn.softmax(weight_array[0])
    prediction_conf = 100 * np.max(score)
    class_type = np.argmax(score)
    print(prediction_conf)

    response = reverse_mapping[class_type]
    print(response)
  else:
    response = {
        "status": "failed"
    }

  return jsonify(response)

if __name__ == '__main__':
    app.run()


 * Serving Flask app '__main__'
 * Debug mode: off

INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit

 * Running on http://025f-35-202-18-247.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040

INFO:werkzeug:127.0.0.1 - - [07/Dec/2023 22:54:57] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [07/Dec/2023 22:54:57] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [07/Dec/2023 22:54:57] "GET /favicon.ico HTTP/1.1" 404 -

1/1 [==============================] - 0s 76ms/step

INFO:werkzeug:127.0.0.1 - - [07/Dec/2023 22:55:06] "POST /api/v1/predict HTTP/1.1" 200 -

7.015844434499741
{'facts': [' Compact size, wrinkled face, and curled tail.', ' Playful and sociable nature, suitable for various households.', ' Prone to weight gain, sensitive to extreme temperatures.'], 'prediction': 'Pug'}



 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
Exception in thread Thread-11:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/urllib3/connection.py", line 199, in _new_conn
    sock = connection.create_connection(
  File "/usr/local/lib/python3.10/dist-packages/urllib3/util/connection.py", line 85, in create_connection
    raise err
  File "/usr/local/lib/python3.10/dist-packages/urllib3/util/connection.py", line 73, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [Errno 111] Connection refused

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py", line 789, in urlopen
    response = self._make_request(
  File "/usr/local/lib/python3.10/dist-packages/urll